In [1]:
import pandas as pd
import pymysql.cursors
import time
from datetime import datetime, timedelta, date

In [2]:
def getconn():
    connection = pymysql.connect(host='{YOUR_HOST_IP}',
                                 user='{YOUR_ID}',
                                 password='{YOUR_PASSWORD}',
                                 db='exchange_rate',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    return connection

def getdata(date):  # %Y-%m-%d
#     headers = {'User-agent' : 'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:22.0) Gecko/20100101 Firefox/22.0'}
    print "loading...", date
    dfs = pd.read_html('http://rate.bot.com.tw/xrt/quote/' + date + '/USD/spot')
    currency = dfs[0]
    currency.columns = [u'時間',u'type',u'現金匯率-本行買入',u'現金匯率-本行賣出',u'即期匯率-本行買入',u'即期匯率-本行賣出']
    return currency

def processdata(currency):
    del currency['type']
    ran = currency.shape[0]
    for r in range(0, ran):
        datalist = []
        for i in currency:
#             print currency[i][r]
            datalist.append(str(currency[i][r]))
        # insert data to DB
        insertdata(datalist)
    connection.commit()
        
def insertdata(datalist):
    with connection.cursor() as cursor:
        # Create a new record
        end = "\"" + datalist[0] + "\"," + datalist[1] + "," + datalist[2] + "," + datalist[3] + "," + datalist[4]
        sql = "INSERT INTO `currency_spot_usd` (`時間`, `現金匯率-本行買入`, `現金匯率-本行賣出`, `即期匯率-本行買入`, `即期匯率-本行賣出`) VALUES ("+end+")"
        cursor.execute(sql)
        
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [3]:
if __name__ == "__main__":
    try:
        start_date = date(2017, 6, 4)
        end_date = date(2017, 12, 31)
        connection = getconn()
        for single_date in daterange(start_date, end_date):
            time.sleep(5)
            currency = getdata(single_date.strftime("%Y-%m-%d"))
            processdata(currency)
    except Exception as e:
        connection.commit()
        print e
    finally:
        connection.close()
        print "scritps finish.", start_date, " to ", single_date

loading... 2017-06-04
scritps finish. 2017-06-04  to  2017-06-04
